MIT License

Copyright (c) 2024 Justin Randall, Smart Interactive Transformations Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Overview

The purpose of this notebook is to serve as basic documentation for using the Interactigen library to generate base and augmented utterance phrases to use with natural language understanding (NLU) intent classification, or with Large Language Model (LLM) fine-tuning.

The Interactigen module is built on LangChain.  The decision to use LangChain enables experimenting with various models with minimal code adjustments.  Already have code to load and configure your favourite LangChain ChatModel?  Great!  Let's use that.

The output results for each run is a JSON array for downstream use: write to a file, split test/train utterances, and more.

## Generate Phrase Utterances with OpenAI

Here is are two examples loading the OpenAI gpt-4-0125-preview and gpt-3.5-turbo-0125 models for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the OPENAI_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
OPENAI_API_KEY=<OpenAI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [6]:
#! pip install dotenv langchain_openai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-4-0125-preview")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240302-gen-phrasing-r3-openai-gpt-4-0125-preview'
)

print(f"{len(phrase_utterances)}")
phrase_utterances

240


['Thank you so much for your help!',
 'I really appreciate your assistance.',
 "You've been incredibly helpful, thanks!",
 'Much obliged for the information.',
 "I can't thank you enough for your support.",
 'Your help has been invaluable, thank you.',
 'Grateful for all your assistance!',
 'Thanks a ton for your guidance.',
 "You've been a great help, cheers!",
 'I appreciate your quick response, thanks.',
 'Your support means a lot, thank you.',
 'Thanks for going above and beyond!',
 'Thank you for your patience and help.',
 "I'm so grateful for your assistance.",
 'Your help has not gone unnoticed, thank you.',
 'Thanks for providing exactly what I needed.',
 'Heartfelt thanks for your guidance.',
 'Thank you for being so helpful!',
 "I appreciate everything you've done, thanks.",
 'Your assistance is greatly appreciated, thank you.',
 'Thanks a bunch!',
 'Really appreciate it.',
 'Super helpful, thanks!',
 'Thanks for the info.',
 "Can't thank you enough.",
 "Your help's been grea

In [7]:
#! pip install dotenv langchain_openai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240302-gen-phrasing-r3-openai-gpt-3.5-turbo-0125'
)

print(f"{len(phrase_utterances)}")
phrase_utterances

278


['Thank you so much!',
 'I really appreciate your help.',
 "You're amazing, bot!",
 'Thanks a bunch!',
 "I'm grateful for your assistance.",
 'Kudos to you, bot!',
 "I can't thank you enough.",
 "You're a lifesaver!",
 'Much obliged!',
 'I owe you one!',
 "You're the best!",
 'Thanks for being so helpful.',
 "I'm so thankful for your support.",
 'You rock, bot!',
 "I'm indebted to you.",
 "I appreciate all you've done.",
 "You've been incredibly helpful.",
 'Big thanks to you!',
 'You deserve all the praise.',
 "I'm so grateful for your guidance.",
 'Thanks a lot!',
 'I appreciate it.',
 "You're great!",
 'Thanks!',
 'Thanks for helping.',
 'Kudos!',
 'Thank you!',
 'You rock!',
 'Thanks!',
 'Thanks!',
 "You're awesome!",
 'Thanks for help.',
 'Thank you!',
 'You rock!',
 'Thank you!',
 'Thank you!',
 "You're helpful.",
 'Big thanks!',
 "You're awesome!",
 'Thank you!',
 'Thank you',
 'I appreciate',
 "You're amazing",
 'Thanks a bunch',
 "I'm grateful",
 'Kudos to you',
 "I can't than

## Generate Phrase Utterances with Google Generative AI

Here is an example loading the Google Generative AI gemini-pro model for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the GOOGLE_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
GOOGLE_API_KEY=<Google Generative AI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [8]:
#! pip install dotenv langchain_google_genai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240302-gen-phrasing-r3-google-gemini-pro'
)

print(f"{len(phrase_utterances)}")
phrase_utterances

228


['Thank you so much!',
 'I appreciate your help.',
 "You've been a lifesaver.",
 'This is incredibly helpful.',
 "You've made my day.",
 "I'm so grateful for your assistance.",
 "You're a lifesaver.",
 "I don't know what I would have done without you.",
 "You're the best!",
 "I'm so impressed with your knowledge.",
 "I'm really grateful for your time.",
 'Thank you for your patience.',
 'I appreciate your help immensely.',
 "You've been a great help.",
 'This has been so informative.',
 "I'm so glad I reached out to you.",
 "You've been so helpful.",
 'I really appreciate your help.',
 'Thank you again for your assistance.',
 'Thanks!',
 'Appreciate it.',
 'Lifesaver.',
 'Helpful.',
 'Made my day.',
 'Grateful.',
 'Lifesaver.',
 "Couldn't have done it without you.",
 'Best!',
 'Impressed.',
 'Grateful.',
 'Thanks for your time.',
 'Appreciate your help.',
 'Great help.',
 'Informative.',
 'Glad I reached out.',
 'So helpful.',
 'Appreciate it.',
 'Thanks again.',
 'Thanks!',
 'Helpful.